In [ ]:
# predictive distribution
def predictive_dis(beta, x, snn, Mn, s, M, n) -> list:
    N = len(x)
    phi = np.zeros([N,M])
    sx = np.zeros(50) # variance of t*
    Mx = np.zeros(50) # mean of t*
   
    
    for m in range(M):
        mu = m / M
        phi[:, m] = np.vectorize(guassian_basis)(x, mu ,s)
    
    for nn in range(N):                            
        sx[nn] = 1.0/beta + np.dot(np.dot(phi[nn, :], snn[n]), phi[nn, :].T)
        Mx[nn] = np.dot(Mn[:,n].T, phi[nn, :].T)
    
    return sx, Mx

# sequence learning
def sequence_learning(x, y ,alpha, beta, s, M):
    N = len(x)
    ssn = np.zeros([N+1, M, M])
    ssn[0, :, :] = linalg.inv(alpha * np.identity(M))
    Mn = np.zeros([M, N+1])
    
    for n in range(N):
        
        xn = np.zeros(n+1)
        xn = x[0:n+1]
        phi = np.zeros([n+1,M])
        
        for m in range(M):
            mu = m / M 
            phi[:, m] = np.vectorize(guassian_basis)(xn, mu ,s)
            
        issn = linalg.solve(ssn[n,: ,:], np.identity(M)) + beta * np.dot(phi.T, phi)
        ssn[n+1,: ,:] = linalg.solve(issn, np.identity(M))
        
        tmp = np.dot(linalg.solve(ssn[n, :, :], np.identity(M)), Mn[:, n]) + beta * np.dot(phi.T, y[:n+1])
        Mn[:, n+1] = np.dot(ssn[n+1], tmp)
    
    return ssn, Mn

# plot the predictive dist. one by one
def predictive_plot(x, y , s, alpha, beta, M) -> list:
    x_new = np.zeros(50)
    y_new = np.zeros(50)
    li = np.arange(50)
    random.shuffle(li)
    pi = 0
    N = len(x)
    sx = np.zeros(50)
    Mx = np.zeros(50)
    
    
    for i in li:
        x_new[pi] = x[i]
        y_new[pi] = y[i]
        pi = pi + 1
    
    [ssn, Mn] = sequence_learning(x_new, y_new, alpha, beta, s, M)
    
    for n in range(50):
#    for n in [50]:
        [sx, Mx] = predictive_dis(beta, x_noise, ssn, Mn, s, M, n)
        
        
        # plot
        plt.figure(n+1)
        plt.plot(x_new, y_new, 'go')
        # plt.plot(x_noise, Mx, 'bo')
        plt.plot(x_noise, Mx, 'blue')
        plt.errorbar(x_noise, Mx, sx ,fmt="o")
        plt.plot(x_curve, y_curve, 'red')
        plt.show()
        print(li[:n+1])
        
    
    return sx, Mx, ssn, Mn

s = 0.5
alpha = 0.01
beta = 20
M2 = 20
sx = np.zeros(50)
Mx = np.zeros(50)

[sx, Mx, ssn_2, Mn_2] = predictive_plot(x_noise, y_noise, s, alpha, beta, M2)